In [ ]:
# label wdpa_pids as (edge_extent_result: sig_increase, sig_decrease, no_change, increase, decrease)

# figure 1: methods figure of transect idea
# figure 2: map of edge_extent_result. (join spatial information)
# figure 2a: sub image of 4 examples 2020 magnitude_gradient image, ndvi_image, raw satellite image
# find most edgey ones but for built, vegetation, elevation, water
# figure 3: pool edge_extent_result by biome and replicate fig3b
# figure 4a: edge_extent ~ AREA_DISSO + gHM_mean + ndvi_mean + elevation_mean + slope_mean + water_extent + (1|biome + year)
#           plot this to show relationship between edge_extent and the covariates
# figure 4b: edge_intensity ~ AREA_DISSO + gHM_mean + ndvi_mean + elevation_mean + slope_mean + water_extent + (1|biome + year)
#           plot this to show relationship between edge_intensity and the covariates
# anova for IUCN_CAT, STATUS_YR, biome, AREA_DISSO to check influence
# figure S1: overall distribution of edge_extent and edge_intensity
# figure S2: correlation plot of edge_intensity vs edge_extent

In [ ]:
# total number of unique WDPA_PID
# total number of transects
# total number of unique ISO3
# total number of unique biome
# total number of WDPA_PID per biome
# percent and number of WDPA_PID edge_extent < 10%
# percent and number of WDPA_PID edge_intensity < 0